In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import copy
import math
import keras 
import rawpy
import random

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM,Embedding,Dropout,Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.python.keras import backend as K
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn

import math
import matplotlib.pyplot as plt
torch.manual_seed(11)

import copy
import math
import keras 
import rawpy
import random

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM,Embedding,Dropout,Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.python.keras import backend as K
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn

import math
import matplotlib.pyplot as plt
torch.manual_seed(11)
import pyspark 
from pyspark.sql import SparkSession

In [2]:

train_path = "./Training"
train_path_fire = train_path+ "/Fire"
train_path_no_fire = train_path+ "/No_Fire"

In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib
matplotlib.use('pdf')

METRICS = ['accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')]

def createModel(baseModel, hidden_layers, num_classes):
    model = Sequential()

    # Transfer Learning
    model.add(baseModel)
    model.layers[0].trainable = True  
    model.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2DTranspose(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(UpSampling2D((2, 2)))   
    model.add(Conv2DTranspose(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
    model.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))
    
    model.add(Flatten())

    # Hidden Layers
    for layer in range(len(hidden_layers)):
        model.add(Dense(hidden_layers[layer], activation='relu'))

    # Add our classification layer and display model properties
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()

    # Compile the sections into one NN
    model.compile(optimizer='sgd', loss='categorical_crossentropy',
                  metrics=METRICS)
    return model


def trainModel(datasetPath, model, epochs, batch_size, image_size, preprocess_input):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)

    train_generator = data_generator.flow_from_directory(
        f'{datasetPath}',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')
    num_training_files = len(train_generator.filepaths)
    
    es_callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
    lrr= ReduceLROnPlateau(monitor='accuracy',   factor=.1,   patience=2,  min_lr=1e-5) 

    # The division by 5 here is to make training quicker
    # when you want maximum accuracy remove it.
    return model.fit(
        train_generator,
        epochs=epochs,
        callbacks=[lrr,es_callback])


def create_pdf(history, model_name):
    print("Generating pdf for " + model_name)

    # grab the desired metrics from tf history
    desired_metrics = ['accuracy', 'loss', 'val_accuracy', 'val_loss']
    metrics_to_plot = dict((k, history.history[k]) for k in desired_metrics if k in history.history)

    df = pd.DataFrame(metrics_to_plot)
    df.index = range(1, len(df)+1)
    df.rename(columns={'accuracy': 'Training Accuracy', 'loss': 'Training Loss',
        'val_accuracy': 'Validation Accuracy', 'val_loss': 'Validation Loss'}, inplace=True)
    print(df)
    sns.set()
    ax = sns.lineplot(marker='o', dashes=False, data=df)
    ax.set_xticks([1, 2, 3, 4, 5])
    ax.set_xlabel('Epoch')
    ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
    ax.set_ylabel('Value')
    ax.set_title('Model Loss and Accuracy')
    plt.savefig(f'./model_statistics/{model_name}_plot.pdf')

    print("Done")
    
def testModel(model, batch_size, datasetPath, num_classes, model_name, image_size, preprocess_input, output_statistics=True):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)

    print("Test" + model_name)
    test_generator = data_generator.flow_from_directory(
        f'./Test',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    num_files = len(test_generator.filepaths)

    steps = num_files/batch_size

    model.evaluate(test_generator, steps=steps)
    if(output_statistics):
        generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)

In [47]:
import lib
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

dataset = "./Training"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'efficientnetb7'
hidden_layers = [128, 64, 32]
num_classes = 2
batch_size = 64
epochs = 1

# Multiclass Settings
# dataset = '/storage/deepfire/subsampledDatasets/forest-1-smoke-fire-forest' # Name of the folder in /storage/deepfire/subsampledDatasets
# num_classes = 3


baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)
#baseModel.summary()
new_model = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )
#new_model.summary()

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

fire_detector_model = createModel(
    new_model, hidden_layers, num_classes)

history = trainModel(dataset, fire_detector_model,
                         epochs, batch_size, image_size, preprocess_input)
create_pdf(history, model_name)



Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_15 (Functional)       (None, 7, 7, 1280)        2257984   
                                                                 
 conv2d_transpose_46 (Conv2D  (None, 9, 9, 16)         184336    
 Transpose)                                                      
                                                                 
 conv2d_transpose_47 (Conv2D  (None, 11, 11, 32)       4640      
 Transpose)                                                      
                                                                 
 up_sampling2d_18 (UpSamplin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_transpose_48 (Conv2D  (None, 24, 24, 64)       18496     
 Transpose)                                          

In [48]:
create_pdf(history, model_name)


Generating pdf for efficientnetb7
   Training Accuracy  Training Loss
1           0.982451       0.042466
Done


In [45]:
testModel(fire_detector_model, batch_size, dataset,
              num_classes, model_name, image_size, preprocess_input, output_statistics)
#fire_detector_model.save(f'saved_models/{model_name}.h5')


Testefficientnetb7
Found 8617 images belonging to 2 classes.
134/134 [==============================] - 20s 139ms/step - loss: 1.5979 - accuracy: 0.6875 - precision: 0.6875 - recall: 0.6875


In [5]:
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Protob Conversion to Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.format("image").option("dropInvalid", True).load(train_path_fire)
df.select("image.origin", "image.width", "image.height").show(truncate=False)



+--------------------------------------------------------------------------------------------------+-----+------+
|origin                                                                                            |width|height|
+--------------------------------------------------------------------------------------------------+-----+------+
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_frame8363.jpg|254  |254   |
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_frame8367.jpg|254  |254   |
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_frame8368.jpg|254  |254   |
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_frame8366.jpg|254  |254   |
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_frame8412.jpg|254  |254   |
|file:///E:/Facultate/Siva_master/Projects/git/change_detection/Training/Fire/resized_fr

In [ ]:
prev_model = tf.keras.applications.EfficientNetV2B1(
    weights=None,
    input_tensor=None,
    input_shape=(45,64,4),
    pooling=None,
    include_preprocessing=True,
)


prev_model = tf.keras.applications.MobileNet(
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(45,64,4),
    pooling=None
)

model = Sequential()
model.add(prev_model)
#prev_model.summary()
model.add(Reshape((4,250,)))
#model.add(Reshape((1,1000,)))
#model.add(Flatten)
x


# ./FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Train

In [6]:
import lib
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

dataset = "./FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Train"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'efficientnetb7'
hidden_layers = [128, 64, 32]
num_classes = 3
batch_size = 64
epochs = 100

# Multiclass Settings
# dataset = '/storage/deepfire/subsampledDatasets/forest-1-smoke-fire-forest' # Name of the folder in /storage/deepfire/subsampledDatasets
# num_classes = 3


baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)
#baseModel.summary()
new_model = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )
#new_model.summary()

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

fire_detector_model = createModel(
    new_model, hidden_layers, num_classes)

history = trainModel(dataset, fire_detector_model,
                         epochs, batch_size, image_size, preprocess_input)
create_pdf(history, model_name)



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_2 (Functional)        (None, 7, 7, 1280)        2257984   
                                                                 
 conv2d_transpose_8 (Conv2DT  (None, 9, 9, 16)         184336    
 ranspose)                                                       
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 11, 11, 32)       4640      
 ranspose)                                                       
                                                                 
 up_sampling2d_4 (UpSampling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 conv2d_transpose_10 (Conv2D  (None, 24, 24, 64)       18496     
 Transpose)                                           

In [7]:
def testModel(model, batch_size, datasetPath, num_classes, model_name, image_size, preprocess_input, output_statistics=True):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)
    print("Test" + model_name)
    test_generator = data_generator.flow_from_directory(
        f'FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Test',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    num_files = len(test_generator.filepaths)

    steps = num_files/batch_size

    model.evaluate(test_generator, steps=steps)
    if(output_statistics):
        generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)

testModel(fire_detector_model, batch_size, dataset,
              num_classes, model_name, image_size, preprocess_input, output_statistics)
#fire_detector_model.save(f'saved_models/{model_name}.h5')


Testefficientnetb7
Found 300 images belonging to 3 classes.
4/4 [==============================] - 3s 516ms/step - loss: 0.8962 - accuracy: 0.7833 - precision: 0.7912 - recall: 0.7833


In [9]:
fire_detector_model.save(f'saved_models/efficientnet_b7_FIRE-SMOKE-DATASET.h5')


# E:\Facultate\Siva_master\Projects\git\change_detection\fire_smoke\Dataset\Training and Validation

In [17]:
import lib
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

dataset = "./fire_smoke/Dataset/Training and Validation"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'efficientnetb7'
hidden_layers = [128, 64, 32]
num_classes = 2
batch_size = 64
epochs = 1

# Multiclass Settings
# dataset = '/storage/deepfire/subsampledDatasets/forest-1-smoke-fire-forest' # Name of the folder in /storage/deepfire/subsampledDatasets
# num_classes = 3


baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)
#baseModel.summary()
new_model = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )
#new_model.summary()

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

fire_detector_model = createModel(
    new_model, hidden_layers, num_classes)

# history = trainModel(dataset, fire_detector_model,
#                          epochs, batch_size, image_size, preprocess_input)
# create_pdf(history, model_name)



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_6 (Functional)        (None, 7, 7, 1280)        2257984   
                                                                 
 conv2d_transpose_24 (Conv2D  (None, 9, 9, 16)         184336    
 Transpose)                                                      
                                                                 
 conv2d_transpose_25 (Conv2D  (None, 11, 11, 32)       4640      
 Transpose)                                                      
                                                                 
 up_sampling2d_12 (UpSamplin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_transpose_26 (Conv2D  (None, 24, 24, 64)       18496     
 Transpose)                                           

In [32]:
def testModel(model, batch_size, datasetPath, num_classes, model_name, image_size, preprocess_input, output_statistics=True):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)
    print("Test" + model_name)
    test_generator = data_generator.flow_from_directory(
        f'fire_smoke/Dataset/Testing',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    num_files = len(test_generator.filepaths)

    steps = num_files/batch_size

    model.evaluate(test_generator, steps=steps)
    if(output_statistics):
        generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)

dataset = "./fire_smoke/Dataset/Training and Validation"
history = trainModel(dataset, fire_detector_model,
                         1, batch_size, image_size, preprocess_input)

testModel(fire_detector_model, batch_size, dataset,
              num_classes, model_name, image_size, preprocess_input, output_statistics)
#fire_detector_model.save(f'saved_models/{model_name}.h5')


Found 1520 images belonging to 2 classes.
24/24 [==============================] - 14s 571ms/step - loss: 0.0028 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - lr: 0.0100
Testefficientnetb7
Found 380 images belonging to 2 classes.
5/5 [==============================] - 1s 136ms/step - loss: 0.1885 - accuracy: 0.9368 - precision: 0.9368 - recall: 0.9368


In [26]:
fire_detector_model.save(f'saved_models/fire_smoke-Dataset-esting_94_acc.h5')


In [34]:
def testModel(model, batch_size, datasetPath, num_classes, model_name, image_size, preprocess_input, output_statistics=True):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)

    print("Test" + model_name)
    test_generator = data_generator.flow_from_directory(
        f'./Test',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    num_files = len(test_generator.filepaths)

    steps = num_files/batch_size

    model.evaluate(test_generator, steps=steps)
    if(output_statistics):
        generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)

dataset = "./Training"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'efficientnetb7'
hidden_layers = [128, 64, 32]
num_classes = 2
batch_size = 64
epochs = 1

history = trainModel(dataset, fire_detector_model,
                         1, batch_size, image_size, preprocess_input)

testModel(fire_detector_model, batch_size, dataset,
              num_classes, model_name, image_size, preprocess_input, output_statistics)
#fire_detector_model.save(f'saved_models/{model_name}.h5')


Found 39375 images belonging to 2 classes.
616/616 [==============================] - 1897s 3s/step - loss: 0.0229 - accuracy: 0.9926 - precision: 0.9926 - recall: 0.9926 - lr: 0.0100
Testefficientnetb7
Found 8617 images belonging to 2 classes.
134/134 [==============================] - 18s 133ms/step - loss: 0.8137 - accuracy: 0.6926 - precision: 0.6926 - recall: 0.6926
